# Functions - Do not delete

In [1]:
def create_dataframe(data_lists, column_names):

    # Create a dictionary with column names as keys and lists as values
    data_dict = {col: data for col, data in zip(column_names, data_lists)}

    # Create DataFrame from the dictionary
    df = pd.DataFrame(data_dict)

    return df


In [2]:
# Get links

import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
import pandas as pd

def create_csv_if_not_exists(filename):
    try:
        with open(filename, 'x') as file:
            pass
    except FileExistsError:
        pass

def scrape_links(start_page, end_page, loop_all=False):
    # base_url = "https://timesofindia.indiatimes.com/topic/Accident"
    base_url = 'https://m.timesofindia.com/topic/car-accident'
    links_set = set()  # Use a set to store unique links
    links_new = []
    create_csv_if_not_exists('link_list.csv')

    with open('link_list.csv', 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            links_set.add(row[0])

    print(f'length of links_set => {len(links_set)}')
    # Determine the range of pages
    if loop_all:
        end_page = 155000  # Set a large number to loop infinitely
        max_value = end_page
    else:
        max_value = end_page

    page_num = start_page

    # Loop until the page number exceeds the end_page

    with tqdm(total=max_value) as pbar:
        # Start the loop
        while page_num <= end_page:
            pbar.update(1)
            url = f"{base_url}/{page_num}"

            try:
                page_request = requests.get(url)
                page_request.raise_for_status()  # Raise an error for HTTP errors
                soup = BeautifulSoup(page_request.content, "html.parser")
                a_tags = soup.select('div.uwU81 a[href]')
                for a_tag in a_tags:
                    link_href = a_tag['href']
                    if link_href not in links_set:  # Check if the link is not already present
                        links_new.append([link_href])  # Add the link to the set
                        links_set.add(link_href)
            except requests.exceptions.RequestException as e:
                print("Error:", e)
                break  # Break the loop if an error occurs

            page_num += 1  # Move to the next page

    print(f'found {len(links_new)} number of links that are new and need to be added')

    # Write new links to CSV file
    with open('link_list.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(links_new)

    print("Links scraped and saved successfully!")

# # Example usage:
# # Scrape links from page 1 to page 4
# scrape_links(1, 10, False)

# # Scrape links from page 1 to all available pages
# # scrape_links(1, 20, loop_all=True)


## Extracting Links/Pages

In [3]:
# Example usage:
# Scrape links from page 1 to page 4
scrape_links(1, 500, False)

# Scrape links from page 1 to all available pages
# scrape_links(1, 20, loop_all=True)

length of links_set => 0


100%|██████████| 500/500 [04:26<00:00,  1.87it/s]

found 9639 number of links that are new and need to be added
Links scraped and saved successfully!


In [10]:
df = pd.read_csv('link_list.csv')
link_list = df.iloc[:,0]

In [11]:
len(link_list)

37

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

# Assuming News_list contains the extracted news content as per your provided code
# I'm assuming you have already initialized News_list as an empty list before using it in your code.

News_list = []
News_date = []

df = pd.read_csv('link_list.csv')
link_list = df.iloc[:,0]

# Your provided code for extracting news content from links
for link_href in tqdm(link_list, desc="Processing links"):
  # soup
  try:
    article_request = requests.get(link_href)
    article_status_code = article_request.status_code
    article_soup = BeautifulSoup(article_request.content, "html.parser")
  except:
    News_date.append("NA")
    News_list.append("NA")
    continue

  # news content
  class_content = article_soup.select_one('._s30J.clearfix')

  if class_content:
    news_content = ' '.join(class_content.stripped_strings).replace('\xa0', ' ')
    News_list.append(news_content)
  else:
    News_list.append("NA")  # Append empty element to the list

  # # News date
  date_element = article_soup.find(class_="xf8Pm byline")
  try:
    date_updated = date_element.span.text
    # pattern = r'Updated:\s+([A-Za-z]+)\s+(\d+),\s+(\d{4}),\s+(\d+:\d+)\s+IST'
    # date_updated = re.match(pattern, date_updated)
    # news_date = date_updated[0]
    News_date.append(date_updated)
  except:
    news_date = 'NA'
    News_date.append(news_date)

Processing links: 100%|██████████| 37/37 [00:13<00:00,  2.72it/s]


In [13]:
print(f'News_list: {len(News_list)}')
print(f'News_date: {len(News_date)}')

News_list: 37
News_date: 37


In [14]:
data = create_dataframe([link_list,News_list,News_date],['link','content','news_date'])

In [15]:
data.to_csv('output/data_road1.csv', index=True)

In [86]:
link = 'https://timesofindia.indiatimes.com/city/delhi/3-die-in-road-accidents-in-delhi-hitandrun-cases/articleshow/107540044.cms'

In [88]:
article_request = requests.get(link)
article_status_code = article_request.status_code
article_soup = BeautifulSoup(article_request.content, "html.parser")

In [97]:
date_element = article_soup.find(class_="xf8Pm byline")

date_updated = date_element.span.text
pattern = r'Updated:\s+([A-Za-z]+)\s+(\d+),\s+(\d{4}),\s+(\d+:\d+)\s+IST'
date_updated = re.match(pattern, date_updated)
# news_date = date_updated[0]
# News_date.append(news_date)

In [98]:
date_updated

In [92]:
date_updated = date_element.span.text